# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
result = establishments.find(query)

# Use count_documents to display the number of documents in the result
print(f"Number of establishments with a hygiene score of 20: {establishments.count_documents(query)}")

# Display the first document in the results using pprint
pprint(result)


Number of establishments with a hygiene score of 20: 41


In [8]:
establishments.count_documents({})

38786

In [9]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))



Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  668fee48f50bc7c39314e96d   110681                  0   
1  668fee48f50bc7c39314ecff   612039                  0   
2  668fee48f50bc7c39314eff7   730933                  0   
3  668fee48f50bc7c39314f1e7   172735                  0   
4  668fee48f50bc7c39314f1ef   172953                  0   
5  668fee48f50bc7c39314fb91   512854                  0   
6  668fee48f50bc7c39314fdc7  1537089                  0   
7  668fee49f50bc7c3931512e1   155648                  0   
8  668fee49f50bc7c39315171f  1012883                  0   
9  668fee49f50bc7c393151f38   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}
result = establishments.find(query)

# Use count_documents to display the number of documents in the result
print(f"Number of establishments in London with RatingValue >= 4: {establishments.count_documents(query)}")

# Display the first document in the results using pprint
pprint(result)

Number of establishments in London with RatingValue >= 4: 33


In [11]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  668fee48f50bc7c393150383   621707                  0   
1  668fee48f50bc7c3931506ad  1130836                  0   
2  668fee49f50bc7c3931511f8   293783                  0   
3  668fee49f50bc7c393151ff5   294474                  0   
4  668fee49f50bc7c393152018  1315095                  0   
5  668fee49f50bc7c39315270b   294900                  0   
6  668fee49f50bc7c393154812   293756                  0   
7  668fee49f50bc7c393154bd7   878523                  0   
8  668fee49f50bc7c393154bed   293772                  0   
9  668fee49f50bc7c393154bee   295107                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000014647              Ti

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [22]:
# # Search within 0.01 degree on either side of the latitude and longitude.
# # Rating value must equal 5
# # Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}
sort = [("scores.Hygiene", 1)]
limit = 5

result = establishments.find(query).sort(sort).limit(limit)

# Print the results
for doc in result:
    pprint(doc)

In [23]:
result

In [25]:
# Get the results from the query and sort
result = list(establishments.find(query).sort(sort).limit(limit))

# Convert result to Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [26]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
result = establishments.aggregate(pipeline)

# Print the number of documents in the result
# Print the first 10 results
print(f"Number of establishments with a hygiene score of 0 in each Local Authority area:")
for doc in result:
    pprint(doc)

Number of establishments with a hygiene score of 0 in each Local Authority area:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}
{'_id': 'Colchester', 'count': 498}
{'_id': 'Tunbridge Wells', 'count': 491}
{'_id': 'Folkestone and Hythe', 'count': 480}
{'_id': 'Eastbourne', 'count': 478}
{'_id': 'Hastings', 'count': 464}
{'_id': 'Bromley', 'count': 460}
{'_id': 'Ashford', 'count': 427}
{'_id': 'Havering', 'count': 397}
{'_id': 'Dartford', 'count': 383}
{'_id': 'Braintree', 'count': 382}
{'_id': 'Basildon', 'count': 362}
{'_id': 'Gravesham', 'count': 339}
{'_id': 'Tonbridge and Malling', 'count': 325}
{'_id': 'Thurrock', 'count': 313}
{'_id': 'Barking and Dagenham', 'count': 301}
{'_id': 'Canterbury 

In [27]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(doc))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in the DataFrame: 2
       0
0    _id
1  count
